In [1]:
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

import time, sys
import datetime
import tqdm.notebook as tqdm
import pandas as pd
import numpy as np

import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

In [5]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
#lets first connect to web driver
#driver=webdriver.Chrome('/Users/apple/Documents/drivers/chromedriver')
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
There is no [mac64] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_mac64.zip
Driver has been saved in cache [/Users/apple/.wdm/drivers/chromedriver/mac64/98.0.4758.102]


In [18]:
driver = webdriver.Chrome(service=s)

In [7]:

url = "https://www.expedia.co.in/Flights"

# Setting the ticket types paths
return_ticket = "//label[@id='flight-type-roundtrip-label-hp-flight']"
one_way_ticket = "//label[@id='flight-type-one-way-label-hp-flight']"
multi_ticket = "//label[@id='flight-type-multi-dest-label-hp-flight']"

In [8]:
def ticket_chooser(ticket):

    try:
        ticket_type = browser.find_element_by_xpath(ticket)
        ticket_type.click()
    except Exception as e:
        pass


In [9]:
# For departure place name
def dep_place_chooser(dep_place):
    fly_from = browser.find_element_by_xpath("//input[@id='flight-origin-hp-flight']")
    time.sleep(1)
    fly_from.clear()
    time.sleep(1.5)
    fly_from.send_keys('  ' + dep_place)
    time.sleep(1.5)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    first_item.click()

In [10]:
# For arrival place name
def arr_place_chooser(arr_place):
    fly_to = browser.find_element_by_xpath("//input[@id='flight-destination-hp-flight']")
    time.sleep(1)
    fly_to.clear()
    time.sleep(1.5)
    fly_to.send_keys('  ' + arr_place)
    time.sleep(1.5)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    first_item.click()

In [11]:
# Departure date
def dep_date_chooser(month, day, year):
    dep_date_button = browser.find_element_by_xpath("//input[@id='flight-departing-hp-flight']")
    dep_date_button.clear()
    dep_date_button.send_keys(month + '/' + day + '/' + year)


In [12]:
# Return date
def return_date_chooser(month, day, year):
    return_date_button = browser.find_element_by_xpath("//input[@id='flight-returning-hp-flight']")

    for i in range(11):
        return_date_button.send_keys(Keys.BACKSPACE)
    return_date_button.send_keys(month + '/' + day + '/' + year)

In [13]:
def search():
    search = browser.find_element_by_xpath("//button[@class='btn-primary btn-action gcw-submit']")
    search.click()
    time.sleep(15)
    print('Results ready!')

In [17]:
df = pd.DataFrame()
def compile_data():
    global df
    global dep_times_list
    global arr_times_list
    global airlines_list
    global price_list
    global durations_list
    global stops_list
    global layovers_list

    #departure times
    dep_times = browser.find_elements_by_xpath("//span[@data-test-id='departure-time']")
    dep_times_list = [value.text for value in dep_times]

    #arrival times
    arr_times = browser.find_elements_by_xpath("//span[@data-test-id='arrival-time']")
    arr_times_list = [value.text for value in arr_times]

    #airline name
    airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
    airlines_list = [value.text for value in airlines]

    #prices
    prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
    price_list =[value.text.split('/div>)[1] for value in prices]

    #durations
    durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
    durations_list = [value.text for value in durations]

    #stops
    stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
    stops_list = [value.text for value in stops]

    #layovers
    layovers = browser.find_elements_by_xpath("//span[@data-test-id='layover-airport-stops']")
    layovers_list = [value.text for value in layovers]

    now = datetime.datetime.now()
    current_date = (str(now.year) + '-' + str(now.month) + '-' + str(now.day))
    current_time = (str(now.hour) + ':' + str(now.minute))
    current_price = 'price' + '(' + current_date + '---' + current_time + ')'
    for i in range(len(dep_times_list)):
        try:
            df.loc[i, 'departure_time'] = dep_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_time'] = arr_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'duration'] = durations_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'stops'] = stops_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'layovers'] = layovers_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, str(current_price)] = price_list[i]
        except Exception as e:
            pass

    print('Comma Separated Value File Created!')

SyntaxError: EOL while scanning string literal (<ipython-input-17-cf31a1772dbe>, line 26)

In [19]:
for i in range(8):    
    driver = webdriver.Chrome(service=s)
    time.sleep(5)

    # choose flights only
    flights_only = driver.find_element_by_xpath("//button[@id='tab-flight-tab-hp']")
    flights_only.click()

    ticket_chooser(return_ticket)

    dep_place_chooser('BOM')

    arr_place_chooser('BLR')

    dep_date_chooser('04', '01', '2022')

    return_date_chooser('05', '02', '2022')

    search()

    compile_data()

    print('run {} completed!'.format(i))

    df.to_csv('Flight_Price_Data.csv')

    time.sleep(15)

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=98.0.4758.109)
Stacktrace:
0   chromedriver                        0x000000010a6ccee9 chromedriver + 5013225
1   chromedriver                        0x000000010a6581d3 chromedriver + 4534739
2   chromedriver                        0x000000010a22ea68 chromedriver + 170600
3   chromedriver                        0x000000010a21e991 chromedriver + 104849
4   chromedriver                        0x000000010a21fe42 chromedriver + 110146
5   chromedriver                        0x000000010a218c12 chromedriver + 80914
6   chromedriver                        0x000000010a22fed3 chromedriver + 175827
7   chromedriver                        0x000000010a29275c chromedriver + 579420
8   chromedriver                        0x000000010a2806d3 chromedriver + 505555
9   chromedriver                        0x000000010a25676e chromedriver + 333678
10  chromedriver                        0x000000010a257745 chrom